# **Cell Type Annotation (CellTypist)**

In [ ]:
import celltypist
from celltypist import models

In [ ]:
# show available models (descriptions)
models.models_description()

In [ ]:
# download a specific model
models.download_models(model='Immune_All_High.pkl')

# or download all available models
# models.download_models()

# By default models are stored under ~/.celltypist/. You can also set a custom location
# using the CELLTYPIST_FOLDER environment variable if needed

# **Prepare adata for Annotation**

In [ ]:
# Create a working copy of normalized data in .X
adata.X = adata.layers["data"].copy()
adata

# **Perform Annotation**

In [ ]:
predictions = celltypist.annotate(adata, model="Immune_All_High.pkl", majority_voting=True)
# Convert to AnnData
annotated_adata = predictions.to_adata()

In [ ]:
# Transfer labels back to adata
adata.obs["celltypist_label"] = annotated_adata.obs.loc[adata.obs_names, "majority_voting"]
adata.obs["celltypist_conf"] = annotated_adata.obs.loc[adata.obs_names, "conf_score"]

In [ ]:
# Visualization
sc.pl.umap(adata, color=["celltypist_label", "celltypist_conf"],
           frameon=False, sort_order=False, wspace=0.3,)

In [ ]:
# Annotated UMAP

# create a figure with width, height, dpi
fig1, ax = plt.subplots(figsize=(8, 6), dpi=600)
# plot onto this ax
sc.pl.umap(adata, color=['celltypist_label'], ax=ax)
# save figure
fig1.savefig("annotated_umap.png", dpi=600, bbox_inches="tight")

# fig 2
sc.pl.umap(
    adata,
    color=["celltypist_label", "leiden"],
    frameon=False,
    sort_order=False,
    wspace=0.3,
    save="./annotated_umap-clusters.png"
)

In [ ]:
# dot plot

predictions.adata.obs['leiden'] = adata.obs['leiden']
# Dotplot (quantitative comparison cluster vs label)
celltypist.dotplot(
    predictions,
    use_as_reference='leiden',            # cluster labels in adata.obs
    use_as_prediction='majority_voting'  # predicted_labels
)

In [ ]:
# export predicted cell types
predictions.to_table(folder='.', prefix='celltypist_')